### Домашнее задание к вебинару 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender

# Метрики
from implicit.evaluation import train_test_split
#from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

from metrics import precision_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
data['week_no'].max()

95

In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [5]:
data_train.shape, data_test.shape

((2278490, 12), (118314, 12))

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [7]:
# Функция, создающая датафрейм items_weights

def items_weights_creator(df):
    
    # Создаем датафрейм, который будет содержать наименование item и количество его покупок
    sales_count = df.groupby('item_id')['quantity'].count().reset_index().sort_values('quantity', ascending=False)
    
    # Переименуем столбец 'user_id'
    sales_count.rename(columns = {'quantity':'n_sold'}, inplace= True)
    
    # добавим столбец weight
    sales_count['weight'] = np.log(sales_count['n_sold'])/sum(np.log(sales_count['n_sold']))
    
    #удалим промежуточный вспомогательный столбец sales_quantity
    sales_count.drop('n_sold', axis = 1, inplace = True)
    
    items_weights = sales_count
    
    return items_weights
    

In [8]:
items_weights_train = items_weights_creator(data_train)

In [9]:
def weighted_random_recommendation(df, n=5):
    
    # добавим дополнительный параметр p, определяющий вероятность элемента оказаться в выборке choice
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

In [10]:
items_weights = items_weights_creator(data_train)

In [11]:
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x:\
                                                                   weighted_random_recommendation(items_weights, n=5))

#### Рассчитаем все остальные baseline

In [12]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [13]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

CPU times: user 2.77 s, sys: 3.52 ms, total: 2.77 s
Wall time: 2.77 s


,user_id,actual,weighted_random_recommendation,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9527185, 9803483, 822583, 1613178, 1062306]","[9305404, 828052, 855326, 1836123, 1348931]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2495324, 936886, 883932, 1056212, 1132911]","[15926887, 12131966, 1025399, 820518, 1076248]"


In [14]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [15]:
%%time

popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

result.head(2)

CPU times: user 53.8 ms, sys: 6.26 ms, total: 60.1 ms
Wall time: 62.6 ms


,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9527185, 9803483, 822583, 1613178, 1062306]","[9305404, 828052, 855326, 1836123, 1348931]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2495324, 936886, 883932, 1056212, 1132911]","[15926887, 12131966, 1025399, 820518, 1076248]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [16]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis = 1).mean(),4)}:{name_col}")

1.0:actual
0.002:weighted_random_recommendation
0.0009:random_recommendation
0.1552:popular_recommendation


**Вывод**: случаные рекомендации и взвешенные случайные рекомендации уступают по уровню precision рекомендациям, сделанным по популярности.

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

#### Рассчитаем байзлайн RandomRecommendation на топ-5000 товаров

In [17]:
#Рассчитаем популярность товаров на основе количества их покупок
popular = data_train.groupby('item_id')['quantity'].sum().reset_index()
popular.rename(columns={'quantity': 'n_sold'}, inplace=True)
popular_5000 = popular.sort_values('n_sold').head(5000).item_id.tolist()

In [18]:
# RandomRecommender

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(popular_5000, n=5))

In [19]:
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9527185, 9803483, 822583, 1613178, 1062306]","[9305404, 828052, 855326, 1836123, 1348931]","[6534178, 6533889, 1029743, 6534166, 1082185]","[975852, 928254, 968415, 5750010, 13775036]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2495324, 936886, 883932, 1056212, 1132911]","[15926887, 12131966, 1025399, 820518, 1076248]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6904451, 6553358, 6467736, 923144, 8293582]"


### Рассчитаем рекомендации с помощьюдетерминированных моделей

#### ItemItemRecommender

In [20]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(popular_5000), 'item_id'] = 999999
data_train.head(100)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,999999,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,999999,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,999999,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,999999,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,999999,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,999999,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,999999,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,5978648,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,999999,1,1.25,288,-0.34,1341,1,0.0,0.0


In [21]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [22]:
user_item_matrix.shape

(2499, 5001)

In [23]:
# id из user_item_matrix
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

#id из sparse матрицы
matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

#словарь для перевода из одних индексов в другие и обратно

# ключи - индексы мактрицы, значения - первоначальные индексы
matrixid_to_itemid = dict(zip(matrix_itemids, itemids))
matrixid_to_userid = dict(zip(matrix_userids, userids))

# ключи -первоначальные индексы, значения - индексы матрицы
itemid_to_matrixid = dict(zip(itemids, matrix_itemids))
userid_to_matrixid = dict(zip(userids, matrix_userids))

In [24]:
# создадим модель
ii_model_5 = ItemItemRecommender(K=5, num_threads = 4)

In [25]:
#обучим модель (ВАЖНО! Подаем в модель транспонированную матрицу user_item!!!!
ii_model_5.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

In [26]:
itemid_to_matrixid[999999]

1033

In [27]:
recs = ii_model_5.recommend(userid=userid_to_matrixid[2],  # userid - id от 0 до N 
                       # делаем предсказание для юзера с индексом 2 - переводим в индекс спарс матрицы индекс 2
                       
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=[itemid_to_matrixid[999999]], 
                        recalculate_user=True)

In [28]:
recs

[(2731, 1075.0), (2728, 883.0), (1913, 18.0), (2843, 7.0), (3204, 1.0)]

In [29]:
[matrixid_to_itemid[rec[0]] for rec in recs]

[5978656, 5978648, 5565043, 5993051, 6424456]

In [30]:
result['ItemItemRec_5'] = result['user_id'].apply(lambda x: [matrixid_to_itemid[rec[0]] for rec in 
                                    ii_model_5.recommend(userid=userid_to_matrixid[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items = [itemid_to_matrixid[999999]], 
                                    recalculate_user=True)])

In [31]:
# Увеличим число соседей в модели до 10

ii_model_10 = ItemItemRecommender(K=10, num_threads = 4)

ii_model_10.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

result['ItemItemRec_10'] = result['user_id'].apply(lambda x: [matrixid_to_itemid[rec[0]] for rec in 
                                    ii_model_10.recommend(userid=userid_to_matrixid[x],
                                                   user_items = sparse_user_item,
                                                   N=5,
                                                   filter_already_liked_items = False,
                                                   filter_items=None,
                                                   recalculate_user = True)])



  0%|          | 0/5001 [00:00<?, ?it/s]

In [32]:
# Увеличим число соседей в модели до 15

ii_model_15 = ItemItemRecommender(K=15, num_threads = 4)

ii_model_15.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

result['ItemItemRec_15'] = result['user_id'].apply(lambda x: [matrixid_to_itemid[rec[0]] for rec in 
                                    ii_model_15.recommend(userid=userid_to_matrixid[x],
                                                   user_items = sparse_user_item,
                                                   N=5,
                                                   filter_already_liked_items = False,
                                                   filter_items=[itemid_to_matrixid[999999]],
                                                   recalculate_user = True)])


  0%|          | 0/5001 [00:00<?, ?it/s]

#### CosineRecommender

In [33]:
cos_model_5 = CosineRecommender(K=5, num_threads = 4)

In [34]:
#обучим модель (ВАЖНО! Подаем в модель транспонированную матрицу user_item!!!!)

cos_model_5.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

In [35]:
recs = cos_model_5.recommend(userid=userid_to_matrixid[2],
                          user_items=sparse_user_item,
                          N=5,
                          filter_already_liked_items = False,
                          filter_items = [itemid_to_matrixid[999999]],
                          recalculate_user = False
                          )

In [36]:
result['CosineRec_5'] = result['user_id'].apply(lambda x: [matrixid_to_itemid[rec[0]] for rec 
                                                         in cos_model_5.recommend(
                                                             userid = userid_to_matrixid[x],
                                                             user_items = sparse_user_item,
                                                             N=5,
                                                             filter_already_liked_items = False,
                                                             filter_items = [itemid_to_matrixid[999999]],
                                                             recalculate_user = False
                                                         )])

In [37]:
# Увеличим число соседей в модели до 10

cos_model_10 = CosineRecommender(K=10, num_threads = 4)

cos_model_10.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

result['CosineRec_10'] = result['user_id'].apply(lambda x: [matrixid_to_itemid[rec[0]] for rec in 
                                    cos_model_10.recommend(userid=userid_to_matrixid[x],
                                                   user_items = sparse_user_item,
                                                   N=5,
                                                   filter_already_liked_items = False,
                                                   filter_items=[itemid_to_matrixid[999999]],
                                                   recalculate_user = True)])




  0%|          | 0/5001 [00:00<?, ?it/s]

In [38]:
# с 15 соседями
cos_model_15 = CosineRecommender(K=15, num_threads = 4)

In [39]:
cos_model_15.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress = True)

  0%|          | 0/5001 [00:00<?, ?it/s]

In [40]:
result['CosineRec_15'] = result['user_id'].apply(lambda x: [matrixid_to_itemid[rec[0]] for rec 
                                                         in cos_model_15.recommend(
                                                             userid = userid_to_matrixid[x],
                                                             user_items = sparse_user_item,
                                                             N=5,
                                                             filter_already_liked_items = False,
                                                             filter_items = [itemid_to_matrixid[999999]],
                                                             recalculate_user = False
                                                         )])

#### TFIDFRecommender

In [41]:
tfidf_model_5 = TFIDFRecommender(K=5, num_threads = 4)

In [42]:
tfidf_model_5.fit(csr_matrix(user_item_matrix).T.tocsr())

  0%|          | 0/5001 [00:00<?, ?it/s]

In [43]:
result['TFIDFRec_5'] = result['user_id'].apply(lambda x: [matrixid_to_itemid[rec[0]] for rec 
                                                         in tfidf_model_5.recommend(
                                                             userid = userid_to_matrixid[x],
                                                             user_items = sparse_user_item,
                                                             N=5,
                                                             filter_already_liked_items = False,
                                                             filter_items = [itemid_to_matrixid[999999]],
                                                             recalculate_user = False
                                                         )])

In [44]:
# Увеличим число соседей в модели до 10

tfidf_model_10 = TFIDFRecommender(K=10, num_threads = 4)

tfidf_model_10.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

result['TFIDFRec_10'] = result['user_id'].apply(lambda x: [matrixid_to_itemid[rec[0]] for rec in 
                                    tfidf_model_10.recommend(userid=userid_to_matrixid[x],
                                                   user_items = sparse_user_item,
                                                   N=5,
                                                   filter_already_liked_items = False,
                                                   filter_items=[itemid_to_matrixid[999999]],
                                                   recalculate_user = True)])





  0%|          | 0/5001 [00:00<?, ?it/s]

In [45]:
# Увеличим количество соседей до 15

tfidf_model_15 = TFIDFRecommender(K=15, num_threads = 4)

In [46]:
tfidf_model_15.fit(csr_matrix(user_item_matrix).T.tocsr())

  0%|          | 0/5001 [00:00<?, ?it/s]

In [47]:
result['TFIDFRec_15'] = result['user_id'].apply(lambda x: [matrixid_to_itemid[rec[0]] for rec 
                                                         in tfidf_model_15.recommend(
                                                             userid = userid_to_matrixid[x],
                                                             user_items = sparse_user_item,
                                                             N=5,
                                                             filter_already_liked_items = False,
                                                             filter_items = [itemid_to_matrixid[999999]],
                                                             recalculate_user = False
                                                         )])

In [48]:
result.head(3)

,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation,random_recommendation_5000,ItemItemRec_5,ItemItemRec_10,ItemItemRec_15,CosineRec_5,CosineRec_10,CosineRec_15,TFIDFRec_5,TFIDFRec_10,TFIDFRec_15
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9527185, 9803483, 822583, 1613178, 1062306]","[9305404, 828052, 855326, 1836123, 1348931]","[6534178, 6533889, 1029743, 6534166, 1082185]","[975852, 928254, 968415, 5750010, 13775036]","[5978656, 5978648, 5565043, 5993051, 6773032]","[999999, 5978656, 5978648, 5565043, 5993051]","[5978656, 5978648, 5565043, 5993051, 5126107]","[5564226, 6773032, 987073, 5978656, 5978648]","[987073, 6773032, 5564226, 5978656, 5978648]","[5564226, 6773032, 987073, 5978656, 5978648]","[6773032, 5564226, 987073, 5978656, 5978648]","[5564226, 6773032, 987073, 5978656, 5978648]","[6773032, 5564226, 987073, 5978656, 5978648]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2495324, 936886, 883932, 1056212, 1132911]","[15926887, 12131966, 1025399, 820518, 1076248]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6904451, 6553358, 6467736, 923144, 8293582]","[5978656, 5978648, 5565043, 5993051]","[999999, 5978656, 5978648, 5565043, 5993051]","[5978656, 5978648, 5565043, 5993051, 5977100]","[5978656, 5978648, 5565043, 5993051]","[5978656, 5978648, 5565043, 5993051, 5977100]","[5978656, 5978648, 5565043, 5993051, 5977100]","[5978656, 5978648, 5565043, 5977100]","[5978656, 5978648, 5565043, 5977100, 5993051]","[5978656, 5978648, 5565043, 5977100, 5993051]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[950659, 934434, 142450, 1539953, 851676]","[1200891, 5995156, 13095123, 1055911, 9677672]","[6534178, 6533889, 1029743, 6534166, 1082185]","[9297519, 5581116, 6634442, 5666448, 6556663]","[5978656, 5978648, 5565043, 5993051, 6633416]","[999999, 5978656, 5978648, 5565043, 5993051]","[5978656, 5978648, 5565043, 5993051, 5126107]","[5978656, 5978648, 6633416, 5565043, 5993051]","[5978656, 5978648, 6633416, 5565043, 5993051]","[5978656, 5978648, 6633416, 5565043, 5993051]","[5978656, 5978648, 6633416, 5565043, 5993051]","[5978656, 5978648, 6633416, 5565043, 5993051]","[5978656, 5978648, 6633416, 5565043, 5993051]"


Рассчитаем теперь precision@5 для всех имеющихся моделей

In [49]:
for col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[col], row['actual']), axis = 1).mean(),4)}:{col}")

1.0:actual
0.002:weighted_random_recommendation
0.0009:random_recommendation
0.1552:popular_recommendation
0.0002:random_recommendation_5000
0.0273:ItemItemRec_5
0.0254:ItemItemRec_10
0.0255:ItemItemRec_15
0.0204:CosineRec_5
0.02:CosineRec_10
0.02:CosineRec_15
0.0202:TFIDFRec_5
0.02:TFIDFRec_10
0.02:TFIDFRec_15


Можно сделать вывод, что с увеличением количества соседей в моделях, метрики меняются несущественно, в целом находятся на одном уровне.  
Наилучший результат дала модель ItemItemRecommender с наименьшим количеством соседей  
Также все модели выдают одинаковые товары, возможно, увеличивая количества соседей, результаты будут различаться.